In [1]:
import pandas as pd
import ktrain
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/j/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec 21 21:27:31 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070        On  | 00000000:01:00.0  On |                  N/A |
|  0%   32C    P8               9W / 200W |    446MiB / 12282MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
data = pd.read_csv('../data/sample/annotated_samples/agreements.csv')

In [3]:
data['annotation'] = data['jake']

In [5]:
data.head()

,Unnamed: 0,message,jake,paige,anton,annotation
0,0,translation : the turkish government has once...,0.0,0.0,0.0,0.0
1,1,"source : pravda gerashchenko translation : ""t...",0.0,0.0,0.0,0.0
2,2,"translation : boris johnsonuk, a glorious cos...",0.0,0.0,0.0,0.0
3,6,source : pravda gerashchenko translation : th...,0.0,0.0,0.0,0.0
4,7,translation : the moment of arrival in kharko...,1.0,1.0,1.0,1.0


In [4]:
sample_amount = len(data[data['annotation']==1.0])

combat  = data[data['annotation']==1.0]

not_combat = data[data['annotation']==0.0].sample(n=sample_amount)

In [5]:
data_sample = pd.concat([combat, not_combat])

In [8]:
data_sample.describe()

,Unnamed: 0,jake,paige,anton,annotation
count,76.000000,76.000000,76.000000,76.000000,76.000000
mean,103.000000,0.500000,0.500000,0.500000,0.500000
std,58.193241,0.503322,0.503322,0.503322,0.503322
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,55.750000,0.000000,0.000000,0.000000,0.000000
50%,109.000000,0.500000,0.500000,0.500000,0.500000
75%,156.500000,1.000000,1.000000,1.000000,1.000000
max,197.000000,1.000000,1.000000,1.000000,1.000000


## Modeling

In [6]:
target_names = ['combat', 'not combat']

Next, I'll load in a pre-trained transformer model. For now, I will use 'distilbert'.

In [26]:
train, val, preprocess = ktrain.text.texts_from_df(
    data_sample,
    text_column = 'message',
    label_columns = ['annotation'],
    val_df = None,
    max_features = 20000,
    maxlen = 128,
    val_pct = 0.1,
    ngram_range = 1,
    preprocess_mode = 'distilbert',
    verbose = 1
)

/home/j/.local/lib/python3.10/site-packages/ktrain/utils.py:768: UserWarning: class_names implies classification but targets array contains float(s) instead of integers or strings
  warnings.warn(
/home/j/.local/lib/python3.10/site-packages/ktrain/utils.py:768: UserWarning: class_names implies classification but targets array contains float(s) instead of integers or strings
  warnings.warn(


['not_annotation', 'annotation']
     not_annotation  annotation
75              1.0         0.0
50              1.0         0.0
83              1.0         0.0
45              1.0         0.0
115             0.0         1.0
['not_annotation', 'annotation']
     not_annotation  annotation
87              0.0         1.0
117             0.0         1.0
65              0.0         1.0
79              0.0         1.0
32              0.0         1.0
preprocessing train...
language: en
train sequence lengths:
	mean : 102
	95percentile : 315
	99percentile : 490


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 59
	95percentile : 94
	99percentile : 98


In [27]:
model = preprocess.get_classifier()
learner = ktrain.get_learner(model, train_data=train, val_data=val, batch_size=32)

Now that the model is all loaded up. I will check which learning rate is the most effective for reducing loss.

In [16]:
learner.lr_find(max_epochs=6)

simulating training for different learning rates... this may take a few moments...
Epoch 1/6


In [ ]:
learner.lr_plot()

ValueError: ignored

In [28]:
history=learner.autofit(lr=1e-4, epochs=10, early_stopping=True)



begin training using onecycle policy with max lr of 0.0001...
3/3 [==============================] - 45s 10s/step - loss: 0.7022 - accuracy: 0.4118 - val_loss: 0.7337 - val_accuracy: 0.1250


In [29]:
predictor = ktrain.get_predictor(learner.model, preproc=preprocess)

In [30]:
validation = learner.validate(val_data=val, print_report=True)

1/1 [==============================] - 3s 3s/step
              precision    recall  f1-score   support

           0       0.12      1.00      0.22         1
           1       0.00      0.00      0.00         7

    accuracy                           0.12         8
   macro avg       0.06      0.50      0.11         8
weighted avg       0.02      0.12      0.03         8

